In [32]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
#import maup
from zipfile import ZipFile
import numpy as np
import os
import shutil

# State House Special Election

In [33]:
pwd

'/Users/peterhorton/Documents/RDH/pdv/pdv-va/vest-va-2016'

In [34]:
final = gpd.read_file('./vest_va_2016/va_2016_statehouse_special/va_2016_statehouse_special.shp')
final.head()

,COUNTYFP,LOCALITY,VTDST,PRECINCT,HOD_DIST,S16HODDEM,S16HODREP,S16HODWRI,geometry
0,095,James City County,000101,Berkeley A,093,1109,1681,14,"POLYGON Z ((-76.82290 37.23270 0.00000, -76.81..."
1,095,James City County,000201,Jamestown A,093,490,409,3,"POLYGON Z ((-76.75462 37.26158 0.00000, -76.75..."
2,095,James City County,000501,Roberts A,093,860,608,3,"POLYGON Z ((-76.65883 37.20304 0.00000, -76.64..."
3,095,James City County,000502,Roberts B,093,914,1727,6,"POLYGON Z ((-76.70027 37.24874 0.00000, -76.70..."
4,095,James City County,000503,Roberts C,093,327,194,3,"POLYGON Z ((-76.68089 37.26080 0.00000, -76.68..."


In [35]:
county_dict = pd.Series(final['COUNTYFP'].values, index = final['LOCALITY']).to_dict()

# Raw Election Results

### District 77

In [36]:
df77 = pd.read_csv('./vest_va_2016/Virginia_Elections_Database__2016_House_of_Delegates_General_Election_District_77_including_precincts.csv')

In [37]:
df77.head()

,County/City,Ward,Pct,"Clifton Eugene Hayes, Jr",All Others,Total Votes Cast
0,NaN,NaN,NaN,Democratic,NaN,NaN
1,Chesapeake City,-,3 - Camelot,"1,831",27.0,"1,858"
2,Chesapeake City,-,5 - Crestwood,"1,394",35.0,"1,429"
3,Chesapeake City,-,8 - South Norfolk Recreation,"1,405",77.0,"1,482"
4,Chesapeake City,-,10 - Oscar Smith,647,47.0,694


In [38]:
df77['County/City'].nunique()

3

In [39]:
df77.shape

(31, 6)

In [40]:
df77 = df77.iloc[1:]
df77 = df77.drop(['Ward'], axis = 1)
df77['HOD_DIST'] = '77'
column_dict = {'County/City': 'LOCALITY', 'Clifton Eugene Hayes, Jr': 'S16HODDEM', 
               'All Others': 'S16HODWRI',
              'Michael Paul Mullin': 'S16HODDEM', 'Heather Lynn Cordasco': 'S16HODREP'
              }

df77['LOCALITY'] = df77['County/City']
df77['S16HODDEM'] = df77['Clifton Eugene Hayes, Jr']
df77['S16HODWRI'] = df77['All Others']
df77 = df77.drop(['County/City', 'Clifton Eugene Hayes, Jr', 'All Others'], axis = 1)

In [41]:
df77.head()

,Pct,Total Votes Cast,HOD_DIST,LOCALITY,S16HODDEM,S16HODWRI
1,3 - Camelot,"1,858",77,Chesapeake City,"1,831",27.0
2,5 - Crestwood,"1,429",77,Chesapeake City,"1,394",35.0
3,8 - South Norfolk Recreation,"1,482",77,Chesapeake City,"1,405",77.0
4,10 - Oscar Smith,694,77,Chesapeake City,647,47.0
5,12 - Georgetown,"1,253",77,Chesapeake City,"1,217",36.0


In [42]:
df77.LOCALITY.nunique()

3

In [43]:
df77['COUNTYFP'] = df77['LOCALITY'].map(county_dict)

In [44]:
county_dict

{'James City County': '095',
 'York County': '199',
 'Chesapeake City': '550',
 'Newport News City': '700',
 'Suffolk City': '800',
 'Williamsburg City': '830'}

In [45]:
df77['LOCALITY'].value_counts()

Chesapeake City    20
Suffolk City        9
TOTALS              1
Name: LOCALITY, dtype: int64

In [46]:
df77.COUNTYFP.nunique()

2

In [47]:
df77.COUNTYFP.value_counts()

550    20
800     9
Name: COUNTYFP, dtype: int64

In [48]:
df77['S16HODDEM'] = df77['S16HODDEM'].map(lambda x: str(x).replace(',', '')).astype(str).astype(float).astype(int)
df77['S16HODWRI'] = df77['S16HODWRI'].map(lambda x: str(x).replace(',', '')).astype(str).astype(float).astype(int)

In [49]:
df77.groupby(['COUNTYFP']).sum()

,S16HODDEM,S16HODWRI
COUNTYFP,,
550,21701,793
800,5415,64


### District 93

In [50]:
df93 = pd.read_csv('./vest_va_2016/Virginia_Elections_Database__2016_House_of_Delegates_General_Election_District_93_including_precincts.csv')

In [51]:
df93.head()

,County/City,Ward,Pct,Michael Paul Mullin,Heather Lynn Cordasco,All Others,Total Votes Cast
0,NaN,NaN,NaN,Democratic,Republican,NaN,NaN
1,James City County,-,101 - Berkeley A,886,"1,383",9.0,"2,278"
2,James City County,-,104- Berkeley D,324,341,1.0,666
3,James City County,-,201 - Jamestown A,391,337,2.0,730
4,James City County,-,501 - Roberts A,687,500,2.0,"1,189"


In [52]:
print(df93['County/City'].nunique())
df93 = df93.iloc[1:]
df93['HOD_DIST'] = '93'
df93['LOCALITY'] = df93['County/City']
df93['S16HODDEM'] = df93['Michael Paul Mullin']
df93['S16HODREP'] = df93['Heather Lynn Cordasco']
df93['S16HODWRI'] = df93['All Others']
df93 = df93.drop(['County/City', 'Michael Paul Mullin', 'Heather Lynn Cordasco', 'All Others', 'Ward'], axis = 1)

df93['COUNTYFP'] = df93['LOCALITY'].map(county_dict)
print(df93['COUNTYFP'].nunique())

df93['S16HODDEM'] = df93['S16HODDEM'].map(lambda x: str(x).replace(',', '')).astype(str).astype(float).astype(int)
df93['S16HODREP'] = df93['S16HODREP'].map(lambda x: str(x).replace(',', '')).astype(str).astype(float).astype(int)
df93['S16HODWRI'] = df93['S16HODWRI'].map(lambda x: str(x).replace(',', '')).astype(str).astype(float).astype(int)

5
4


In [53]:
df93.COUNTYFP.value_counts()

700    11
095     9
199     5
830     4
Name: COUNTYFP, dtype: int64

In [54]:
df93.LOCALITY.value_counts()

Newport News City    11
James City County     9
York County           5
Williamsburg City     4
TOTALS                1
Name: LOCALITY, dtype: int64

In [55]:
df93.groupby(['COUNTYFP']).sum()

,S16HODDEM,S16HODREP,S16HODWRI
COUNTYFP,,,
095,5296,6499,31
199,1615,3148,14
700,8641,5475,35
830,4865,2512,17


In [56]:
df93.shape

(30, 8)

In [57]:
final.HOD_DIST.value_counts()

077    25
093    21
Name: HOD_DIST, dtype: int64

## Districts 77 and 93 together

In [58]:
elections = []
elections.append(df77)
elections.append(df93)
df = pd.concat(elections, axis = 0)

df = df[df['LOCALITY'] != 'TOTALS']

df = df.reset_index()

In [59]:
column_list = ['S16HODDEM', 'S16HODWRI', 'S16HODREP']

In [60]:
for val in column_list:
    vote_dif = df[val].sum()-final[val].sum()
    if (vote_dif == 0):
        print(val+": EQUAL")
    else:
        print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES")

S16HODDEM: EQUAL
S16HODWRI: EQUAL
S16HODREP: EQUAL


In [61]:
print("Counties with differences printed below:")
diff_counties=[]
for i in column_list:
    diff = df.groupby(['COUNTYFP']).sum()[i]-final.groupby(['COUNTYFP']).sum()[i]
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(diff[diff != 0].to_string(header=False))
print("")
print("All other races in all counties are equal")

Counties with differences printed below:

All other races in all counties are equal


## Reallocate Absentee Votes

In [62]:
df93.COUNTYFP.value_counts()

700    11
095     9
199     5
830     4
Name: COUNTYFP, dtype: int64

In [63]:
df77.COUNTYFP.value_counts() #no overlap in counties so no need to incorporate district to reallocate

550    20
800     9
Name: COUNTYFP, dtype: int64

In [64]:
df = df.fillna(0)

In [65]:
def get_absentee_frac(prec_tot, county_tot, absentee_tot):
    '''A spec specifies in first line what function will do
    Returns fraction of county absentee votes for a precinct for a candidate
    
    Keyword arguments: (put inputs to funtions and say what they should be)
    precinct vote total for given candidate (int)
    county vote total for given candidate, absentee and provisional votes not included (int)
    absentee vote total for county for given candidate (int)
    
    "Pure function" bc only relies on inputs and outputs - only variables are ones that were passed as arguments
    '''
    if county_tot != 0:
        prec_frac = prec_tot/county_tot
        absentee_frac = prec_frac*absentee_tot
    else:
        absentee_frac = 0
    return absentee_frac

In [66]:
absentee_and_prov = df[(df['Pct'].map(lambda x: 'Absentee' in str(x))) | (df['Pct'].map(lambda x: 'Provisional' in str(x)))]
absentee_and_prov

,index,Pct,Total Votes Cast,HOD_DIST,LOCALITY,S16HODDEM,S16HODWRI,COUNTYFP,S16HODREP
18,19,Ab - Central Absentee Precinct,"2,939",77,Chesapeake City,2867,72,550,0.0
19,20,Provisional,16,77,Chesapeake City,16,0,550,0.0
27,28,Ab - Central Absentee Precinct,877,77,Suffolk City,868,9,800,0.0
28,29,Provisional,3,77,Suffolk City,3,0,800,0.0
36,8,Ab - Central Absentee Precinct,"2,222",93,James City County,1066,9,095,1147.0
37,9,Provisional,6,93,James City County,1,1,095,4.0
47,19,Ab - Central Absentee Precinct,"1,145",93,Newport News City,776,3,700,366.0
48,20,Provisional,13,93,Newport News City,6,0,700,7.0
51,23,Ab - Central Absentee Precinct,898,93,Williamsburg City,545,0,830,353.0
52,24,Provisional,12,93,Williamsburg City,6,0,830,6.0


In [67]:
absentee_and_prov.COUNTYFP.nunique()

6

In [68]:
df.COUNTYFP.nunique()

6

In [69]:
groupby_absentee_and_prov_tot = absentee_and_prov.groupby(['COUNTYFP']).sum()
groupby_absentee_and_prov_tot

,index,S16HODDEM,S16HODWRI,S16HODREP
COUNTYFP,,,,
095,17,1067,10,1151.0
199,57,204,2,296.0
550,39,2883,72,0.0
700,39,782,3,373.0
800,57,871,9,0.0
830,47,551,0,359.0


In [70]:
groupby_county_df_tot = df.groupby(['COUNTYFP']).sum()
df_no_absent_or_provisional = df[(df['Pct'].map(lambda x: 'Absentee' not in str(x))) & (df['Pct'].map(lambda x: 'Provisional' not in str(x)))
                                & (df['LOCALITY'] != 'TOTALS')]
groupby_county_tot_no_absentee = df_no_absent_or_provisional.groupby('COUNTYFP').sum()

In [72]:
df_with_absentee_reallocated = df_no_absent_or_provisional.copy()
groupby_absentee_and_prov_tot.reset_index(inplace=True,drop=False)
groupby_county_tot_no_absentee.reset_index(inplace=True,drop=False)

to_dole_out_totals = groupby_absentee_and_prov_tot.copy()
precinct_specific_totals = groupby_county_tot_no_absentee.copy()

## PH CODE for vote allocation

#countyfp_cd
#df_no_absent_or_provisional

#Create some new columns for each of these races to deal with the allocation
for race in column_list:
    add_var = race+"_add"
    rem_var = race+"_rem"
    floor_var = race+"_floor"
    df_with_absentee_reallocated.loc[:,add_var]=0.0
    df_with_absentee_reallocated.loc[:,rem_var]=0.0
    df_with_absentee_reallocated.loc[:,floor_var]=0.0

#Iterate over the rows
#Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
for index, row in df_no_absent_or_provisional.iterrows():
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #Grab the district
        county_id = row['COUNTYFP']
        #Get the denominator for the allocation (the precinct vote totals)
        denom = precinct_specific_totals.loc[precinct_specific_totals['COUNTYFP']==county_id][race]
        #Get one of the numerators, how many districtwide votes to allocate
        numer = to_dole_out_totals.loc[to_dole_out_totals['COUNTYFP']==county_id][race]
        #Get the vote totals for this race in this precinct
        val = df_with_absentee_reallocated.at[index,race]
        #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
        if ((float(denom)==0)):
            vote_share = 0
        else:
            vote_share = (float(val)/float(denom))*float(numer)
        df_with_absentee_reallocated.at[index,add_var] = vote_share
        #Take the decimal remainder of the allocation
        df_with_absentee_reallocated.at[index,rem_var] = vote_share%1
        #Take the floor of the allocation
        df_with_absentee_reallocated.at[index,floor_var] = np.floor(vote_share)

#After the first pass through, get the sums of the races by district to assist in the rounding            
first_allocation = pd.DataFrame(df_with_absentee_reallocated.groupby(['COUNTYFP']).sum())

#Now we want to iterate district by district to work on rounding
county_list = list(to_dole_out_totals['COUNTYFP'].unique()) 

#Iterate over the district
for county in county_list:
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
        to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals['COUNTYFP']==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
        #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
        for index in df_with_absentee_reallocated.loc[df_with_absentee_reallocated['COUNTYFP']==county][rem_var].nlargest(to_go).index:
            df_with_absentee_reallocated.at[index,add_var] = np.ceil(df_with_absentee_reallocated.at[index,add_var])

#Iterate over every race again
for race in column_list:
    add_var = race+"_add"
    #Round every allocation down to not add fractional votes
    df_with_absentee_reallocated.loc[:,add_var]=np.floor(df_with_absentee_reallocated.loc[:,add_var])
    df_with_absentee_reallocated.loc[:,race]+=df_with_absentee_reallocated.loc[:,add_var]

In [73]:
for val in column_list:
    vote_dif = df_with_absentee_reallocated[val].sum()-final[val].sum()
    if (vote_dif == 0):
        print(val+": EQUAL")
    else:
        print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES")
        
        
print("Counties with differences printed below:")
diff_counties=[]
for i in column_list:
    diff = df_with_absentee_reallocated.groupby(['LOCALITY']).sum()[i]-final.groupby(['LOCALITY']).sum()[i]
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(diff[diff != 0].to_string(header=False))
print("")
print("All other races in all counties are equal")

S16HODDEM: EQUAL
S16HODWRI: EQUAL
S16HODREP: EQUAL
Counties with differences printed below:

All other races in all counties are equal


In [ ]:
df_with_absentee_reallocated = df_no_absent_or_provisional.copy()
for index, precinct_total_row_by_candidate in df_no_absent_or_provisional.iterrows():
    #print(precinct_total_row_by_candidate)
    county_fips = precinct_total_row_by_candidate['COUNTYFP']
    county_total_row_by_candidate_no_absentee = groupby_county_tot_no_absentee.loc[county_fips]
    county_absentee_and_provisional_by_candidate = groupby_absentee_and_prov_tot.loc[county_fips]
    #print(precinct_total_row_by_candidate, county_total_row_by_candidate_no_absentee, county_absentee_and_provisional_by_candidate)
    for candidate, votes in county_total_row_by_candidate_no_absentee.to_dict().items():
        prec_tot = precinct_total_row_by_candidate[candidate]
        #print('precinct total: ', prec_tot)
        county_tot = votes
        #print('county total: ', county_tot)
        absentee_tot = county_absentee_and_provisional_by_candidate[candidate]
        #print('absentee total: ', absentee_tot)
        absentee_frac = get_absentee_frac(prec_tot, county_tot, absentee_tot)
        #print('absentee votes to allocate to precinct: ', absentee_frac)
        prec_with_absentee_allocation = round(prec_tot + absentee_frac)
        #print('new precinct total: ', prec_with_absentee_allocation)
        df_with_absentee_reallocated.loc[index, candidate] = round(prec_with_absentee_allocation) #index is row, candidate is column

In [ ]:
df_with_absentee_reallocated.shape

In [ ]:
final.shape

## Check column sums

In [ ]:
df_with_absentee_reallocated.columns

In [ ]:
column_list = ['S16HODDEM', 'S16HODWRI', 'S16HODREP']

In [ ]:
groupby_county_final = final.groupby('COUNTYFP').sum()
groupby_df_with_absentee_reallocated = df_with_absentee_reallocated.groupby('COUNTYFP').sum()
df_absentee_reallocate_with_final = pd.merge(groupby_df_with_absentee_reallocated, groupby_county_final, on = 'COUNTYFP', how = 'inner', suffixes = ('_x', '_y'))

In [ ]:
def sum_checker(df, column_list):
    list_max = 0
    for i in column_list:
        left_sum = df[i+'_x'].sum()
        right_sum = df[i+'_y'].sum()
        if (left_sum == right_sum):
            print('column: ', i, ' have the same sums')
        
        else:
            diff = abs(left_sum - right_sum)
            percent = abs(left_sum - right_sum)/left_sum
            print('column: ', i)
            print('RDH sum is ', str(left_sum))
            print('Partner sum is ', str(right_sum))
            print('their (absolute) difference is ', str(diff))
            print('As a percentage of DF1 this is ', str(round(percent*100,5)) + '%')
            if (percent>list_max):
                list_max = percent
            print('')
    print('The max difference (as a percent of the total of on DF column is: ', str(round(list_max,5)))

In [ ]:
sum_checker(df_absentee_reallocate_with_final, column_list)

In [ ]:
(df_with_absentee_reallocated.COUNTYFP + df_with_absentee_reallocated.Pct.str.slice(stop = 3)).nunique()

In [ ]:
(final.COUNTYFP + final.VTDST.str.slice(start = 3)).nunique()

### Create unique id to enable merge

In [ ]:
set(df_with_absentee_reallocated.COUNTYFP + df_with_absentee_reallocated.Pct.str.slice(stop = 3)) - set(final.COUNTYFP + final.VTDST.str.slice(start = 3))

In [ ]:
def vtdst_changer(vtdst):
    if (vtdst[1:3] == ' -'):
        two_lead_zero = '00' + vtdst[:1]
        return two_lead_zero
    elif (vtdst[1:3] == '- '):
        two_lead_zero = '00' + vtdst[:1]
        return two_lead_zero
    elif (vtdst[-1:] == ' '):
        one_lead_zero = '0' + vtdst[:2]
        return one_lead_zero
    elif (vtdst[-1:] == '-'):
        one_lead_zero = '0' + vtdst[:2]
        return one_lead_zero
    else:
        return vtdst

In [ ]:
df_with_absentee_reallocated['vtdst'] = df_with_absentee_reallocated.Pct.str.slice(stop = 3)
df_with_absentee_reallocated['vtdst'] = df_with_absentee_reallocated['vtdst'].apply(vtdst_changer)
df_with_absentee_reallocated.head()

In [ ]:
df_with_absentee_reallocated['unique_id'] = df_with_absentee_reallocated['COUNTYFP'] + df_with_absentee_reallocated['vtdst']
df_with_absentee_reallocated.head()

In [ ]:
final['unique_id'] = final.COUNTYFP + final.VTDST.str.slice(start = 3)

In [ ]:
print(set(df_with_absentee_reallocated.unique_id) - set(final.unique_id))
print(set(final.unique_id) - set(df_with_absentee_reallocated.unique_id))

# Shapefile

In [ ]:
county_fips = []
for directory in os.listdir('./vest_va_2016/census_shps_by_county_all_unzip/'):
    if not directory[0] == '.':
        county_fips.append(directory[-5:])
        
proj = final.crs   

county_vtds = []
for i in county_fips: #i dont have fips_codes file
    ref = './vest_va_2016/census_shps_by_county_all_unzip/partnership_shapefiles_19v2_'
    vtd_ref = ref + i + '/PVS_19_v2_vtd_' + i + '.shp' 
    vtd_shp = gpd.read_file(vtd_ref)
    print(i, ' added to list')
    county_vtds.append(vtd_shp)

global shp_vtd
shp_vtd = gpd.GeoDataFrame(pd.concat(county_vtds, axis = 0) , crs = proj) 

county_sldl = []
for i in county_fips: #i dont have fips_codes file
    ref = './vest_va_2016/census_shps_by_county_all_unzip/partnership_shapefiles_19v2_'
    sldl_ref = ref + i + '/PVS_19_v2_sldl_' + i + '.shp' #PVS_19_v2_sldl_51650
    sldl_shp = gpd.read_file(sldl_ref)
    print(i, ' added to list')
    county_sldl.append(sldl_shp)

global shp_sldl
shp_sldl = gpd.GeoDataFrame(pd.concat(county_sldl, axis = 0) , crs = proj) 

overlay = gpd.overlay(shp_sldl, shp_vtd, how='intersection', make_valid=True, keep_geom_type=True)
shp = overlay[(overlay.SLDLST == '077') | (overlay.SLDLST == '093')]

In [ ]:
shp.plot()
final.plot()

In [ ]:
shp.shape

In [ ]:
final.shape

In [ ]:
shp.columns

In [ ]:
print((set(shp.COUNTYFP_1 + shp.VTDST.str.slice(start = 3)) - set(final.COUNTYFP + final.VTDST.str.slice(start = 3))))
print((set(final.COUNTYFP + final.VTDST.str.slice(start = 3)) - set(shp.COUNTYFP_1 + shp.VTDST.str.slice(start = 3))))

In [ ]:
final[final.unique_id == '095104']

In [ ]:
shp['unique_id'] = shp['COUNTYFP_1'] + shp['VTDST'].str.slice(start = 3)

In [ ]:
shp[(shp['unique_id'] == '095041') | (shp['unique_id'] == '095042')].plot()
final[final['unique_id'] == '095104'].plot()

In [ ]:
id_095_dict = {'095041': '095104', '095042': '095104'}

In [ ]:
shp['unique_id_1'] = shp['unique_id']
shp.loc[shp['unique_id'].isin(id_095_dict.keys()), 'unique_id'] = shp.loc[shp['unique_id'].isin(id_095_dict.keys()), 'unique_id'].map(id_095_dict)
shp.loc[shp['unique_id_1'].isin(id_095_dict.keys())]

In [ ]:
shp = shp.dissolve(by = 'unique_id', as_index = False)

In [ ]:
shp.loc[shp['unique_id_1'].isin(id_095_dict.keys())]

In [ ]:
print((set(shp.unique_id) - set(final.unique_id)))
print((set(final.unique_id) - set(shp.unique_id)))

In [ ]:
final[final['unique_id'].isin(['199401', '800601', '700211', '700108'])].plot()

In [ ]:
set(final.unique_id) - set(df_with_absentee_reallocated.unique_id)

In [ ]:
final[final['unique_id'].isin(['199401', '800601', '700211', '700108'])]

### Join election results and shapefile

In [ ]:
join = gpd.GeoDataFrame(pd.merge(df_with_absentee_reallocated, shp, on = 'unique_id', how = 'outer'), crs = proj)
join.plot()
final.plot()

In [ ]:
final

# Validation

## Validating Shapefile

Here, I merge the shp joined to the election results to the final file to compare, rather than the raw shp to final as was done is previous VA files...

In [ ]:
shp_final_merge = pd.merge(join, final, on = 'unique_id', how = 'outer', suffixes = ['_x', '_y'],indicator=True)

join['geometry'].geom_almost_equals(final['geometry'], decimal = 0)

shp_shapes = gpd.GeoDataFrame(shp_final_merge, geometry = 'geometry_x')
final_shapes = gpd.GeoDataFrame(shp_final_merge, geometry = 'geometry_y')
vals = shp_shapes.geom_almost_equals(final_shapes, decimal = 0)

In [ ]:
both = shp_final_merge[shp_final_merge["_merge"]=="both"]
source_geoms = gpd.GeoDataFrame(both,geometry="geometry_x",crs=final.crs)
vest_geoms = gpd.GeoDataFrame(both,geometry="geometry_y",crs=final.crs)
source_geoms = source_geoms.to_crs(3857)
vest_geoms = vest_geoms.to_crs(3857)
source_geoms["geometry_x"]=source_geoms.buffer(0)
vest_geoms["geometry_y"]=vest_geoms.buffer(0)
vals = source_geoms.geom_almost_equals(vest_geoms,decimal=0)
print(vals.value_counts())

In [ ]:
count = 0
area_list = []
big_diff = pd.DataFrame(columns=["area"])
for i in range(0,len(source_geoms)):
    diff = source_geoms.iloc[[i]].symmetric_difference(vest_geoms.iloc[[i]])
    intersection = source_geoms.iloc[[i]].intersection(vest_geoms.iloc[[i]])
    area = float(diff.area/10e6)
    area_list.append(area)
    #print("Area is " + str(area))

    if (area > 1):
        count += 1
        name = source_geoms.at[i,"Pct"]
        big_diff.loc[name]=area
        print(str(count)+") For " + name + " difference in area is " + str(area))
        if (intersection.iloc[0].is_empty):
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            base.set_title(name)
        else:
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            intersection.plot(color="green",ax=base)
            base.set_title(name)

In [ ]:
df = pd.DataFrame(area_list)
print(df.shape)

print(str(len(df[df[0]==0]))+" precincts w/ a difference of 0 km^2")
print(str(len(df[(df[0]<.1) & (df[0]>0)]))+ " precincts w/ a difference between 0 and .1 km^2")
print(str(len(df[(df[0]<.5) & (df[0]>=.1)]))+ " precincts w/ a difference between .1 and .5 km^2")
print(str(len(df[(df[0]<1) & (df[0]>=.5)]))+ " precincts w/ a difference between .5 and 1 km^2")
print(str(len(df[(df[0]<2) & (df[0]>=1)]))+ " precincts w/ a difference between 1 and 2 km^2")
print(str(len(df[(df[0]<5) & (df[0]>=2)]))+ " precincts w/ a difference between 2 and 5 km^2")
print(str(len(df[(df[0]>=5)]))+ " precincts w/ a difference greater than 5 km^2")

In [ ]:
print(vals.value_counts())

In [ ]:
print(shp_final_merge[~vals]['unique_id'].str[0:3].value_counts())
diff_precincts = gpd.GeoDataFrame(shp_final_merge[~vals], geometry = 'geometry_x', crs = proj)

shp_difference = gpd.overlay(shp_shapes, final_shapes, how = 'symmetric_difference')
shp_difference.plot()

In [ ]:
#Show a random difference
test = shp_final_merge[~vals].sample(axis = 0, n=1).index[0]
print(shp_final_merge[~vals])
display(shp_final_merge.iloc[test]['geometry_x'])
display(shp_final_merge.iloc[test]['geometry_y'])

test = shp_final_merge[~vals].sample(axis = 0).index[0]
display(shp_final_merge.iloc[test]['geometry_x'])
display(shp_final_merge.iloc[test]['geometry_y'])

### County 550 example

In [ ]:
print(shp_final_merge[shp_final_merge['unique_id'].str[0:3] == '550'].shape)
ex_x = gpd.GeoDataFrame(shp_final_merge[shp_final_merge['unique_id'].str[0:3]== '550'], geometry = 'geometry_x', crs = proj)
ex_y = gpd.GeoDataFrame(shp_final_merge[shp_final_merge['unique_id'].str[0:3]== '550'], geometry = 'geometry_y', crs = proj)

gpd.overlay(ex_x, ex_y, how = 'symmetric_difference').plot()

## Validating Election Results

In [ ]:
final.columns

In [ ]:
column_list = ['S16HODDEM', 'S16HODREP', 'S16HODWRI']

In [ ]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff >0):
                if(diff>0):
                    print(i, "{:.>72}".format(df.iloc[j]["Pct"]), "(V)","{:.>5}".format(int(df.iloc[j][right_Data]))," (S){:.>5}".format(int(df.iloc[j][left_Data])),"(D):{:>5}".format(int(df.iloc[j][right_Data])-int(df.iloc[j][left_Data])))           
                #print(df.iloc[j]['countypct'])
                
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]["Pct"])
                    print(df.iloc[j][left_Data])
                    print(df.iloc[j][right_Data])
                if (diff>max_diff):
                    max_diff = diff
                    #print("New max diff is: ", str(max_diff))
                    #print(df.iloc[j]['cty_pct'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]["Pct"])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    diff_list.sort()
    print(diff_list)

In [ ]:
final_join_merge = pd.merge(join, final, on = 'unique_id', how = 'inner', suffixes = ['_x', '_y'])

In [ ]:
validater_row(final_join_merge, column_list)